In [1]:
using DynamicPolynomials

@ncpolyvar x[1:2]
f = 2.0+x[1]^2+x[1]*x[2]+x[2]*x[1]+x[2]^2
g = [1.0-x[1]^2-x[2]^2]
#h = Vector{Polynomial{false,Float64}}([])
h = [x[1]*x[2]+x[2]*x[1]+1.0]

1-element Array{Polynomial{false,Float64},1}:
 x₁x₂ + x₂x₁ + 1.0

In [14]:
using JuMP, Mosek

model = Model(with_optimizer(Mosek.Optimizer, QUIET=false))
G0=@variable(model, [1:3,1:3], PSD)

typeof(G0)

LinearAlgebra.Symmetric{VariableRef,Array{VariableRef,2}}

In [4]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

lm, supp, coe=ctpNCPOP.info(f,x,2)

(5, Array{UInt64,1}[[0x0000000000000001, 0x0000000000000001], [0x0000000000000001, 0x0000000000000002], [0x0000000000000002, 0x0000000000000002], [0x0000000000000002, 0x0000000000000001], []], [1.0, 1.0, 1.0, 1.0, 2.0])

In [10]:
using SparseArrays
supp=sparse([5 0;0 1;0 0])
unique(supp.rowval)

2-element Array{Int64,1}:
 1
 2

In [22]:
ctpNCPOP._sym_canon(Vector{UInt64}([2;3;1]))

3-element Array{UInt64,1}:
 0x0000000000000001
 0x0000000000000003
 0x0000000000000002

In [24]:
A=[1; 2]
B=[1; 2]

A==B

true

In [161]:
@ncpolyvar x[1:2]
v=reverse(monomials(x,0:2))

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end

v=reverse(monomials(x,0:2))
v+=star_algebra.(v)
v=v./2

7-element Array{Polynomial{false,Float64},1}:
 1.0              
 x₂               
 x₁               
 0.5x₁x₂ + 0.5x₂x₁
 x₂²              
 0.5x₁x₂ + 0.5x₂x₁
 x₁²              

In [159]:
star_algebra(v[1])

ArgumentError: ArgumentError: reducing over an empty collection is not allowed

In [149]:
v[17]

x₁²x₃x₂

In [150]:
ind=v[17].z .>0

3-element BitArray{1}:
 1
 1
 1

In [153]:
ind

3-element BitArray{1}:
 1
 1
 1

In [151]:
vars=v[17].vars[ind]

3-element Array{PolyVar{false},1}:
 x₁
 x₃
 x₂

In [152]:
exp=v[17].z[ind]

3-element Array{Int64,1}:
 2
 1
 1

In [154]:
prod(vars[i]^exp[i] for i in length(exp):-1:1)

x₂x₃x₁²

In [117]:
k=2
println("Relaxation order: k=",k)

Relaxation order: k=2


In [118]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h,sparse=false);

In [119]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val1=ctpNCPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                                 maxit=Int64(1e6),tol=1e-3,
                                 use_eqcons_to_get_constant_trace=false,
                                 check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=2
  Size of the largest block: s^max=7
  Number of equality trace constraints: zeta=

19
Modeling time:
  0.129656 seconds (153.99 k allocations: 7.798 MiB)
iter=1   val=-5.887846720064157e-17   gap=5.357819291094374e-17   feas=2.190972608948529
iter=2   val=0.1084770103411627   gap=0.5219975339610938   feas=1.8757430557536698
iter=4   val=1.7947554537973327   gap=1.249051612408578   feas=0.5123625241106545
iter=8   val=1.742144500141759   gap=1.0248897823639014   feas=0.452140292586313
iter=16   val=1.7283210967697304   gap=0.585241557273317   feas=0.30066718006710363
iter=32   val=1.850231571082291   gap=0.36787980665967623   feas=0.16441235381368594
iter=64   val=2.0078396591295067   gap=0.026654888707663877   feas=0.07427882335106706
iter=128   val=1.97839146978514   gap=0.12898422162116957   feas=0.03768912162327496
iter=256   val=2.0062088333365704   gap=0.04662224530463259   feas=0.01779490644401029
iter=512   val=1.9949638100054596   gap=0.004786263070253096   feas=0.009614209985758139
iter=1024   val=2.0017886267613587   gap=0.033500972983920185   feas=0.004255

1.9995613656703728

In [120]:
using NCTSSOS

opt,data = nctssos_first([f; g; h], x, k, numeq=1, TS=false, obj="eigen");

------------------------------------------------------
The sizes of blocks:
[7]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 22              
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - numbe